
# Geography and Logistics Analysis

---

## 📊 BI Questions & Hypotheses Mapping

| BI Question | Related Hypothesis | Business Insight |
| --- | --- | --- |
| Where are most customers located? | H3 | Regional demand concentration analysis |
| What is the median delivery distance and delay time? | H3, H4 | Logistics & operational efficiency |
| Which regions have the worst delivery delays? | H1, H3 | Service quality risk zones |
| Do longer distances correlate with lower review scores? | H1 | Impact on satisfaction & churn |

### Hypotheses:

- **H1:** Delivery delays negatively affect customer satisfaction and reduce repeat purchases.
- **H3:** Delivery distance and geographic factors contribute to delivery delays.
- **H4:** Operational inefficiencies lead to opportunity cost and lost revenue.

---

## 💰 Dollars & Cents Framing

| Scenario | Metric | Estimated $$$ Impact |
| --- | --- | --- |
| Lost revenue from extreme delays (>20 days) | $110,688/year | Direct revenue at risk |
| Long distance shipping cost (North/NW) | ~2500km average | Potential $200K/year in extra shipping costs |
| Customer retention loss (due to late delivery churn) | CLV: $100-$500 per customer | Retain up to $300K/year in repeat purchases |
| Warehouse optimization | Distance reduction | Potential $300K/year in logistics savings |

---

## 🔬 Hypothesis Testing & Discussion

### H1 (Delays impact satisfaction): **Partially Supported**
- Correlation between delays and review score: weak (-0.05).
- Extreme delays caused ~$110,688 in lost revenue.
- CLV churn loss estimated ~$300K/year.

### H3 (Distance impacts delays): **Weakly Supported**
- Correlation between distance and delay: weak (-0.03).
- North/NW regions have avg distance ~2500km and face ~20 days delays.

### H4 (Operational inefficiencies drive revenue loss): **Strongly Supported**
- Total business opportunity ~$700K/year combining revenue loss, shipping cost, CLV churn, and optimization savings.

---

## 📈 Executive Takeaway

By addressing extreme delays, optimizing delivery networks, and improving seller compliance, the business can recover ~$700K/year in revenue opportunities while improving customer satisfaction.

---


# 📊 Action Plan — Geography & Logistics

---

## 🔧 Objective:
Reduce delivery delays, improve customer satisfaction, and recover revenue by addressing operational inefficiencies.

---

## 1️⃣ Build Regional Hub (North/NW)

- ✅ **Problem:** North/NW regions (RR, AC, AM) have median delivery distances ~2500km.
- ✅ **Impact:** Extreme delays >20 days causing ~$110K/year in lost revenue.
- ✅ **Action:** Build or partner with a warehouse closer to these regions.
- ✅ **Investment:** ~$500K CAPEX (one-time).
- ✅ **Return:** Save ~$200K/year in reduced long-haul shipping costs.

---

## 2️⃣ Route Optimization System

- ✅ **Problem:** Existing logistics are not efficiently optimized for high-distance orders.
- ✅ **Impact:** Average delay ~20 days in worst regions.
- ✅ **Action:** Implement AI-based route optimization algorithms to improve delivery SLA.
- ✅ **Cost:** ~$50K software implementation.
- ✅ **Savings:** ~$100K/year in operational efficiency (fuel, routes, time).

---

## 3️⃣ Seller Compliance Monitoring

- ✅ **Problem:** Delays often occur at seller fulfillment stage.
- ✅ **Impact:** Late seller dispatches contribute to overall delivery failure.
- ✅ **Action:** Implement real-time SLA monitoring for sellers.
- ✅ **Cost:** Minimal (utilize existing order tracking data).
- ✅ **Revenue Protected:** ~$100K/year via reduced delivery failures.

---

## 4️⃣ Customer Recovery Program

- ✅ **Problem:** Late deliveries drive churn and lost Customer Lifetime Value.
- ✅ **Impact:** ~$300K/year at risk due to churned customers.
- ✅ **Action:** Launch automated customer recovery offers (e.g. coupons, loyalty perks).
- ✅ **Cost:** ~$20K/year program budget.
- ✅ **Revenue Recovered:** ~$300K/year in retained CLV.

---

## 💵 High-Level Summary

| Initiative | Cost | Annual Benefit |
| --- | --- | --- |
| Regional Hub (North/NW) | $500K (one-time) | $200K/year |
| Route Optimization | $50K | $100K/year |
| Seller Monitoring | Minimal | $100K/year |
| Customer Recovery | $20K/year | $300K/year |

---

**Total Business Opportunity:** ~$700K/year upside.

---


# 📎 Data Dependency Appendix

This analysis relies on the following data tables and fields:

| Table | Key Fields Used | Purpose in Analysis |
| --- | --- | --- |
| customers | customer_id, customer_unique_id, customer_state, customer_zip_code_prefix | Identify customer location |
| sellers | seller_id, seller_zip_code_prefix, seller_state | Identify seller location |
| geolocation_version1 | geolocation_zip_code_prefix, geolocation_lat, geolocation_lng | Convert customer and seller zip prefixes to latitude & longitude |
| orders_corrected | order_id, customer_id, order_delivered_customer_date, order_estimated_delivery_date | Calculate delivery delays |
| order_items_corrected | order_id, seller_id | Link orders to sellers |
| order_payments | order_id, payment_value | Calculate revenue per order |
| order_reviews | order_id, review_score | Capture customer satisfaction rating |
| Derived Fields | distance_km, delay_days | Calculated delivery distance (Haversine formula) and delivery delay |


Connect to Big Queryfrom google.cloud import bigquery

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

client = bigquery.Client(project="brazilian-e-commerce-team-3")

query_sales = """
WITH customer_geo AS (
    SELECT
        c.customer_id,
        c.customer_unique_id,
        c.customer_state,
        AVG(g.geolocation_lat) AS customer_lat,
        AVG(g.geolocation_lng) AS customer_lng
    FROM
        `brazilian-e-commerce-team-3.BET_Team3.customers` c
    JOIN
        `brazilian-e-commerce-team-3.BET_Team3.geolocation_version1` g
    ON
        c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
    GROUP BY
        c.customer_id, c.customer_unique_id, c.customer_state
),

seller_geo AS (
    SELECT
        s.seller_id,
        s.seller_zip_code_prefix,
        s.seller_state,
        AVG(g.geolocation_lat) AS seller_lat,
        AVG(g.geolocation_lng) AS seller_lng
    FROM
        `brazilian-e-commerce-team-3.BET_Team3.sellers` s
    JOIN
        `brazilian-e-commerce-team-3.BET_Team3.geolocation_version1` g
    ON
        s.seller_zip_code_prefix = g.geolocation_zip_code_prefix
    GROUP BY
        s.seller_id, s.seller_zip_code_prefix, s.seller_state
)

SELECT
    o.order_id,
    cg.customer_unique_id,
    cg.customer_state,
    p.payment_value,
    r.review_score,
    DATE_DIFF(o.order_delivered_customer_date, o.order_estimated_delivery_date, DAY) AS delay_days,
    ST_DISTANCE(
        ST_GEOGPOINT(sg.seller_lng, sg.seller_lat),
        ST_GEOGPOINT(cg.customer_lng, cg.customer_lat)
    ) / 1000 AS distance_km
FROM
    `brazilian-e-commerce-team-3.BET_Team3.orders_corrected` o
JOIN
    `brazilian-e-commerce-team-3.BET_Team3.order_items_corrected` i ON o.order_id = i.order_id
JOIN
    seller_geo sg ON i.seller_id = sg.seller_id
JOIN
    customer_geo cg ON o.customer_id = cg.customer_id
JOIN
    `brazilian-e-commerce-team-3.BET_Team3.order_payments` p ON o.order_id = p.order_id
JOIN
    `brazilian-e-commerce-team-3.BET_Team3.order_reviews` r ON o.order_id = r.order_id
"""

sales = client.query(query_sales).to_dataframe()

#  check
print(sales.head())



In [ ]:
#Section 1 : Customer Distribution
#Where are most customers located?
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

client = bigquery.Client(project="brazilian-e-commerce-team-3")

# Query customer states directly
query_customers = """
SELECT customer_state FROM `brazilian-e-commerce-team-3.BET_Team3.customers`
"""
customers = client.query(query_customers).to_dataframe()

# Plot customer distribution
customer_state_counts = customers['customer_state'].value_counts()
customer_state_counts.plot(kind='bar', figsize=(10,5), title='Customer Distribution by State')
plt.show()

In [ ]:
#Section 2 : Median Delivery Distance
# What is the median delivery distance?
query_sales = """
WITH geolocation_deduped AS (
  SELECT 
    geolocation_zip_code_prefix,
    AVG(geolocation_lat) AS geolocation_lat,
    AVG(geolocation_lng) AS geolocation_lng
  FROM 
    `brazilian-e-commerce-team-3.BET_Team3.dim_geolocation`
  GROUP BY 
    geolocation_zip_code_prefix
),
customer_coords AS (
  SELECT 
    c.customer_id,
    c.customer_state,
    g.geolocation_lat AS customer_latitude,
    g.geolocation_lng AS customer_longitude
  FROM 
    `brazilian-e-commerce-team-3.BET_Team3.customers` c
  JOIN 
    geolocation_deduped g
  ON 
    c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
),
seller_coords AS (
  SELECT 
    s.seller_id,
    g.geolocation_lat AS seller_latitude,
    g.geolocation_lng AS seller_longitude
  FROM 
    `brazilian-e-commerce-team-3.BET_Team3.sellers` s
  JOIN 
    geolocation_deduped g
  ON 
    s.seller_zip_code_prefix = g.geolocation_zip_code_prefix
)
SELECT 
    f.*,
    c.customer_state,
    ST_DISTANCE(
      ST_GEOGPOINT(c.customer_longitude, c.customer_latitude),
      ST_GEOGPOINT(s.seller_longitude, s.seller_latitude)
    ) / 1000 AS distance_km
FROM 
    `brazilian-e-commerce-team-3.BET_Team3.fact_sales_corrected` f
JOIN customer_coords c
  ON f.customer_id = c.customer_id
JOIN seller_coords s
  ON f.seller_id = s.seller_id
"""

# Run query & load into pandas dataframe
sales = client.query(query_sales).to_dataframe()

# Group and plot median delivery distance
state_distances = sales.groupby('customer_state')['distance_km'].median().sort_values(ascending=False)
state_distances.plot(kind='barh', figsize=(10,8), title='Median Delivery Distance by State')
plt.show()


In [ ]:
#Section 3 - Delivery Delays
# Which regions have the worse delivery delays?
# Compute delivery delay
sales['order_delivered_customer_date'] = pd.to_datetime(sales['order_delivered_customer_date'])
sales['order_estimated_delivery_date'] = pd.to_datetime(sales['order_estimated_delivery_date'])
sales['delay_days'] = (sales['order_delivered_customer_date'] - sales['order_estimated_delivery_date']).dt.days

# Group and plot delivery delays
state_delays = sales.groupby('customer_state')['delay_days'].mean().sort_values(ascending=False)
state_delays.plot(kind='barh', figsize=(10,8), title='Average Delivery Delay by State')
plt.show()

In [ ]:
#Section 4 - Correlation Checks
# Correlation between distance & delivery delay
correlation = sales[['distance_km','delay_days']].corr()
print("Correlation between distance and delay_days:")
print(correlation)

# Correlation between distance & review score
correlation_review = sales[['distance_km','review_score']].corr()
print("Correlation between distance and review_score:")
print(correlation_review)


In [ ]:
#Section 5 - Dollars and Cents Framing
#Calculate lost revenue from extreme delays
extreme_delays = sales[sales['delay_days'] > 20]
lost_revenue = extreme_delays['payment_value'].sum()

print(f"Lost Revenue from extreme delays (>20 days): ${lost_revenue:,.2f}")


In [ ]:
# Define North/NW states
north_states = ['RR', 'AC', 'AM']

# Filter sales for those states
north_sales = sales[sales['customer_state'].isin(north_states)]

# Calculate average distance in these states
avg_distance_north = north_sales['distance_km'].mean()

# Estimate extra shipping cost
shipping_cost_per_km = 0.25
total_shipments = len(north_sales)
extra_shipping_cost = avg_distance_north * shipping_cost_per_km * total_shipments

print(f"Estimated extra shipping cost (North/NW): ${extra_shipping_cost:,.2f}")


In [ ]:
# Filter extreme delays
extreme_delays = sales[sales['delay_days'] > 20]

# Calculate customer_lost
customers_lost = extreme_delays['customer_id'].nunique()

# Assume 5% churn rate due to extreme delays
churn_rate = 0.05
lost_customers = customers_lost * churn_rate

# Customer Lifetime Value assumption
CLV_per_customer = 300
total_CLV_loss = lost_customers * CLV_per_customer

print(f"Estimated CLV Loss due to churn: ${total_CLV_loss:,.2f}")


In [ ]:
# Potential distance reduction savings
distance_saving_per_order = 1000  # km reduction
savings_per_order = distance_saving_per_order * shipping_cost_per_km
total_savings = savings_per_order * total_shipments

print(f"Potential warehouse optimization savings: ${total_savings:,.2f}")


In [ ]:

extreme_delays = sales[sales['delay_days'] > 20]
lost_revenue = extreme_delays['payment_value'].sum()
print(f"Lost Revenue from extreme delays (>20 days): ${lost_revenue:,.2f}")


# 📎 Data Dependency Appendix

This analysis relies on the following data tables and fields:

| Table | Key Fields Used | Purpose in Analysis |
| --- | --- | --- |
| customers | customer_id, customer_unique_id, customer_state, customer_zip_code_prefix | Identify customer location |
| sellers | seller_id, seller_zip_code_prefix, seller_state | Identify seller location |
| geolocation_version1 | geolocation_zip_code_prefix, geolocation_lat, geolocation_lng | Convert zip prefixes to lat/long coordinates |
| orders_corrected | order_id, customer_id, order_delivered_customer_date, order_estimated_delivery_date | Calculate delivery delays |
| order_items_corrected | order_id, seller_id | Link sellers to orders |
| order_payments | order_id, payment_value | Capture revenue per order |
| order_reviews | order_id, review_score | Capture customer satisfaction |
| Derived Fields | distance_km, delay_days | Calculated during SQL query using Haversine formula and date differences |

---

✅ **Critical Join Dependencies:**
- `customer_zip_code_prefix` → used to map customers to geo coordinates.
- `seller_zip_code_prefix` → used to map sellers to geo coordinates.
- If any of these fields were missing, delivery distance calculations would fail.

✅ **Note on Data Validation Feedback:**
- While additional fields in dim_products and dim_sellers may be missing for broader BI reporting, they do not affect this geolocation analysis.
